In [8]:
from dotenv import load_dotenv
load_dotenv()
import os

from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [15]:
# Create a Pinecone index
pc = Pinecone(api_key = os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': 'c7d9c3b7a3df43e359c46a0fcf504d61', 'Date': 'Fri, 23 Aug 2024 01:28:22 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [ ]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. John Smith',
  'subject': 'Introduction to Psychology',
  'stars': 4,
  'review': 'Dr. Smith is engaging and makes the material easy to understand. However, his exams can be a bit tricky.'},
 {'professor': 'Dr. Emily Davis',
  'subject': 'Calculus I',
  'stars': 5,
  'review': 'Dr. Davis is fantastic! She explains complex concepts clearly and is very approachable.'},
 {'professor': 'Professor Michael Johnson',
  'subject': 'Modern American History',
  'stars': 3,
  'review': "The subject matter is interesting, but Professor Johnson's lectures can be a bit dry and monotonous."},
 {'professor': 'Dr. Sarah Lee',
  'subject': 'Organic Chemistry',
  'stars': 2,
  'review': 'Dr. Lee is knowledgeable but her teaching style is very fast-paced. I struggled to keep up with the course.'},
 {'professor': 'Professor Daniel Brown',
  'subject': 'Introduction to Sociology',
  'stars': 4,
  'review': 'Professor Brown is very insightful and encourages class participation. The assign

In [14]:

# This format is compatible with OpenAI's text-embedding-ada-002, but may be different for other embedding models
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model='text-embedding-ada-002'
    )
    embedding = response.data[0].embedding
    processed_data.append({``
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })

In [10]:
processed_data[0]

{'values': [-0.008619316,
  0.02319831,
  0.014768502,
  -0.033327147,
  -0.002520774,
  0.027315192,
  -0.012860359,
  -0.03748324,
  -0.029223336,
  -0.017473882,
  -0.0110567715,
  0.0330919,
  -0.010533993,
  0.03087009,
  0.024570605,
  0.0021482941,
  0.033170316,
  -0.012651247,
  0.0018150227,
  -0.016846547,
  -0.045272645,
  0.009148629,
  -0.016075449,
  -0.001012884,
  -0.011305092,
  0.013258978,
  0.010801917,
  -0.00937081,
  -0.01977411,
  0.0060119564,
  0.01182787,
  0.0030043444,
  -0.025812203,
  -0.037221853,
  -0.0068941456,
  0.014350278,
  -0.014781571,
  0.0034176665,
  0.0029978098,
  0.00078743557,
  0.0064301793,
  -0.0074430634,
  0.009834777,
  -0.011050237,
  -0.010122305,
  -0.015604948,
  -0.0020845805,
  -0.03528757,
  -0.041979138,
  0.0036561843,
  0.005172243,
  0.010566667,
  0.00015366838,
  -0.0119651,
  -0.00046886725,
  -0.009527644,
  0.013618388,
  0.009566852,
  -1.5427463e-06,
  -0.022884643,
  -0.010037353,
  -0.013670666,
  -0.0027315193,

In [1]:
index = pc.Index("rag")  # In Firebase, this is your collection
index.upsert(
    vectors=processed_data,
    namespace="ns1", # And this would be your documents
)

NameError: name 'pc' is not defined

In [12]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}